In [1]:
import pandas as pd
import re


In [2]:
data = pd.read_csv(r"G:\work\logistica\input\Dec_cha/Dec_JDSF_cha.csv")

d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,ORDER_ID,LOGISTICS_COMPANY,MAIL_NO,DESCRIPTION1,ROUTE_TIME1,DESCRIPTION2,ROUTE_TIME2,DESCRIPTION3,ROUTE_TIME3,DESCRIPTION4,...,DESCRIPTION36,ROUTE_TIME36,DESCRIPTION37,ROUTE_TIME37,DESCRIPTION38,ROUTE_TIME38,DESCRIPTION39,ROUTE_TIME39,DESCRIPTION40,ROUTE_TIME40
0,690000000001008318121220668130,shunfeng,252149455218,顺丰速运 已收取快件,2018-12-12 18:31,"快件在【南京建邺集庆门大街营业点】已装车,准备发往 【南京江宁集散中心】",2018-12-12 20:52,快件已发车,2018-12-12 21:07,快件到达 【南京江宁集散中心】,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,690000000001008318121220668127,shunfeng,245451608345,顺丰速运 已收取快件,2018-12-12 15:36,"快件在【广州白云鹤龙一路营业部】已装车,准备发往下一站",2018-12-12 21:25,快件已发车,2018-12-12 21:59,快件到达 【广州穗北集散中心】,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,690000000001008318121220668122,jingdongwuliu,VA49330022862,配送员柏旭已经揽收完成,2018-12-12 17:27,货物已交付京东物流,2018-12-12 17:27,货物已完成分拣，离开【太原分拨中心】,2018-12-13 01:53,货物已到达【吕梁柳林营业部】,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,690000000001008318121220668123,shunfeng,245451620838,顺丰速运 已收取快件,2018-12-12 15:41,"快件在【广州白云鹤龙一路营业部】已装车,准备发往下一站",2018-12-12 21:25,快件已发车,2018-12-12 21:59,快件到达 【广州穗北集散中心】,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,690000000001008318121220668021,shunfeng,245449471676,顺丰速运 已收取快件,2018-12-12 14:26,"快件在【广州白云鹤龙一路营业部】已装车,准备发往下一站",2018-12-12 16:29,快件已发车,2018-12-12 17:28,快件到达 【广州穗北集散中心】,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def get_del_list(n):
    '''选出路径时间列'''
    dayslist = []
    for i in range(1, n + 1)[::-1]: # 倒序
        route_time = "ROUTE_TIME" + str(i)
        dayslist.append(route_time)
    return dayslist


In [ ]:
dayslist = get_del_list(40)

In [ ]:
df = data.copy().fillna("kong")
df.drop(dayslist,axis=1, inplace=True)

In [ ]:
print(df["LOGISTICS_COMPANY"].value_counts())

In [ ]:
JD = df[df["LOGISTICS_COMPANY"] == "jingdongwuliu"]

In [ ]:
def get_carrier_list(carrier):
    '''将DataFrame行转换为list'''
    a = []
    for indexs in carrier.index:
        b = carrier.loc[indexs].values[0:-1]
        c = ' '.join('%s' %id for id in b)
        a.append(c)
    return a

In [ ]:
JD_list = get_carrier_list(JD)

In [ ]:
JD_list


In [ ]:
def get_JD_name(JD):
    JD_name_list = []
    for i in JD:
        JD_name_dict = {}
        #deliveryman_JD = re.findall('(\\d+)\\s(\\w+)\\s(\\w+).*配送员，(\\w+)，手机号，(\\d+).*(货物已由\\w+|货物已\\w+)',i)
        #JD_name_dict["JD_OrderId"] = str(re.findall("(\\d+)\\sjingdongwuliu",i))        

        JD_name_dict["JD_OrderId"] = str(re.findall("(\\d+)\\sjingdongwuliu",i)).replace("['","").replace("']","")
        JD_name_dict["JD_trackingID"] = str(re.findall("jingdongwuliu\\s(\\w+)",i)).replace("['","").replace("']","")
        JD_name_dict["JD_deliveryman_name"] = str(re.findall("配送员\，(\\w+)\，手机号\，\\d+\s货物已",i)).replace("['","").replace("']","")
        JD_name_dict["JD_deliveryman_tel"] = str(re.findall("配送员\，\\w+\，手机号\，(\\d+)\s货物已",i)).replace("['","").replace("']","")
        JD_name_dict["JD_orderstatus"] = str(re.findall("\\d+\s(货物已由\\w+|货物已\\w+)，感谢您选择京东物流！",i)).replace("['","").replace("']","")
        JD_name_dict["JD_LastStation"] = str(re.findall("\【(\\w+)\】\s货物已分配\，等待配送",i)).replace("['","").replace("']","")
#         ttt = "【"
#         if ttt in i:
#             JD_name_dict["JD_LastStation"] = re.findall("【(.+?)】",i)[-1]
        JD_name_list.append(JD_name_dict)
        print(JD_name_dict["JD_LastStation"])
    return JD_name_list

In [ ]:
JD_name = get_JD_name(JD_list)
JD_name

In [ ]:
str1 =  '690000000001008318121220658082 jingdongwuliu VD50167410576 货物已交付京东物流 配送员张坤已经揽收完成 货物已完成分拣，离开【成都青白江分拣中心】 货物已完成分拣，离开【泸州中转场】 配送员开始配送，请您准备收货，配送员，何涛，手机号，18180283626 货物已由本人签收，感谢您选择京东物流！ kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong',
str1

In [ ]:
str2 =  '690000000001008318121220657914 jingdongwuliu V048761509790 货物已交付京东物流 配送员宋龙军已经揽收完成 货物已到达【潍坊中转场】 货物已到达【潍坊中转场】 货物已完成分拣，离开【潍坊中转场】 货物已到达【潍坊青云山营业部】 货物已分配，等待配送 配送员开始配送，请您准备收货，配送员，张建军，手机号，13210765501 货物已由本人签收，感谢您选择京东物流！ kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong kong'
str2

In [ ]:
aaa= re.findall("【(\\w+)】\s['货物已分配','配送员开始配送']",str1)